Source:
https://www.tensorflow.org/lite/performance/post_training_integer_quant

# Post traininf Integer Quantizaton
### Overview

Integer quantization is an optimization strategy that converts 32-bit floating-point numbers (such as weights and activation outputs) to the nearest 8-bit fixed-point numbers. This results in a smaller model and increased inferencing speed, which is valuable for low-power devices such as microcontrollers. This data format is also required by integer-only accelerators such as the Edge TPU.

In this tutorial, you'll train an MNIST model from scratch, convert it into a Tensorflow Lite file, and quantize it using [post-training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization). Finally, you'll check the accuracy of the converted model and compare it to the original float model.

You actually have several options as to how much you want to quantize a model. In this tutorial, you'll perform "full integer quantization," which converts all weights and activation outputs into 8-bit integer data—whereas other strategies may leave some amount of data in floating-point.

To learn more about the various quantization strategies, read about [TensorFlow Lite model optimization](https://www.tensorflow.org/lite/performance/model_optimization).




## Setup

In [3]:
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)

import tensorflow as tf
import numpy as np
print("TensorFlow version: ", tf.__version__)


2023-11-06 10:23:51.755047: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-06 10:23:51.930242: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-06 10:23:51.930277: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-06 10:23:51.931369: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-06 10:23:52.031666: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-06 10:23:52.032975: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

TensorFlow version:  2.14.0


### Generate a TensorFlow Model
We'll build a simple model to classify numbers from the [MNIST dataset](https://www.tensorflow.org/datasets/catalog/mnist).

This training won't take long because you're training the model for just a 5 epochs, which trains to about ~98% accuracy.

In [7]:
# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1 
train_images = train_images.astype(np.float32) /255.0
test_images = test_images.astype(np.float32)/255.0

# Define the model architecture
model = tf.keras.Sequential([
  tf.keras.layers.InputLayer(input_shape=(28, 28)),
  tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
  tf.keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['accuracy'])
model.fit(
  train_images,
  train_labels,
  epochs=5,
  validation_data=(test_images, test_labels)
)

Epoch 1/5


2023-11-06 10:26:51.311065: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


1863/1875 [============================>.] - ETA: 0s - loss: 0.3096 - accuracy: 0.9115

2023-11-06 10:26:58.619963: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 31360000 exceeds 10% of free system memory.


1875/1875 [==============================] - 8s 4ms/step - loss: 0.3087 - accuracy: 0.9117 - val_loss: 0.1546 - val_accuracy: 0.9584
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1283 - accuracy: 0.9632 - val_loss: 0.0925 - val_accuracy: 0.9736
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0869 - accuracy: 0.9751 - val_loss: 0.0777 - val_accuracy: 0.9765
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0689 - accuracy: 0.9793 - val_loss: 0.0686 - val_accuracy: 0.9787
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0586 - accuracy: 0.9824 - val_loss: 0.0624 - val_accuracy: 0.9780


## Convert to a TensorFlow Lite model

Now you can convert the trained model to TensorFlow Lite format using the TensorFlow Lite [Converter](https://www.tensorflow.org/lite/models/convert), and apply varying degrees of quantization.

Beware that some versions of quantization leave some of the data in float format. So the following sections show each option with increasing amounts of quantization, until we get a model that's entirely int8 or uint8 data. (Notice we duplicate some code in each section so you can see all the quantization steps for each option.)

First, here's a converted model with no quantization:




In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpjjs9me9_/assets


INFO:tensorflow:Assets written to: /tmp/tmpjjs9me9_/assets
2023-11-06 10:28:43.456288: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-06 10:28:43.456304: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-06 10:28:43.456691: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpjjs9me9_
2023-11-06 10:28:43.457478: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-06 10:28:43.457487: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpjjs9me9_
2023-11-06 10:28:43.458997: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled
2023-11-06 10:28:43.459557: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-06 10:28:43.482435: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

It's now a TensorFlow Lite model, but it's still using 32-bit float values for all parameter data.



## Convert using dynamic range quantization
Now let's enable the default ``optimizations`` flag to quantize all fixed parameters (such as weights):



The model is now a bit smaller with quantized weights, but other variable data is still in float format.



### Convert using float fallback quantization
To quantize the variable data (such as model input/output and intermediates between layers), you need to provide a [`RepresentativeDataset`](https://www.tensorflow.org/api_docs/python/tf/lite/RepresentativeDataset). This is a generator function that provides a set of input data that's large enough to represent typical values. It allows the converter to estimate a dynamic range for all the variable data. (The dataset does not need to be unique compared to the training or evaluation dataset.) To support multiple inputs, each representative data point is a list and elements in the list are fed to the model according to their indices.




Now all weights and variable data are quantized, and the model is significantly smaller compared to the original TensorFlow Lite model.

However, to maintain compatibility with applications that traditionally use float model input and output tensors, the TensorFlow Lite Converter leaves the model input and output tensors in float:




That's usually good for compatibility, but it won't be compatible with devices that perform only integer-based operations, such as the Edge TPU.

Additionally, the above process may leave an operation in float format if TensorFlow Lite doesn't include a quantized implementation for that operation. This strategy allows conversion to complete so you have a smaller and more efficient model, but again, it won't be compatible with integer-only hardware. (All ops in this MNIST model have a quantized implementation.)

So to ensure an end-to-end integer-only model, you need a couple more parameters...

### Convert using integer-only quantization

To quantize the input and output tensors, and make the converter throw an error if it encounters an operation it cannot quantize, convert the model again with some additional parameters:



The internal quantization remains the same as above, but you can see the input and output tensors are now integer format:



Now you have an integer quantized model that uses integer data for the model's input and output tensors, so it's compatible with integer-only hardware such as the [Edge TPU](https://coral.ai/).



### Save the models as files
You'll need a `.tflite` file to deploy your model on other devices. So let's save the converted models to files and then load them when we run inferences below.

### Run the TensorFlow Lite models
Now we'll run inferences using the TensorFlow Lite [``Interpreter``](https://www.tensorflow.org/api_docs/python/tf/lite/Interpreter) to compare the model accuracies.

First, we need a function that runs inference with a given model and images, and then returns the predictions:




### Test the models on one image
Now we'll compare the performance of the float model and quantized model:

* `tflite_model_file` is the original TensorFlow Lite model with floating-point data.
* `tflite_model_quant_file` is the last model we converted using integer-only quantization (it uses uint8 data for input and output).

Let's create another function to print our predictions:




Now test the float model:



And test the quantized model:



# Evaluate the models on all images
Now let's run both models using all the test images we loaded at the beginning of this tutorial:

Evaluate the float model:



Evaluate the quantized model:



So you now have an integer quantized a model with almost no difference in the accuracy, compared to the float model.



To learn more about other quantization strategies, read about [TensorFlow Lite model optimization](https://www.tensorflow.org/lite/performance/model_optimization).



https://www.tensorflow.org/lite/performance/post_training_integer_quant